# API Extract CVD Prevent Data

In [7]:
from itertools import product
import pandas as pd
import requests
import time

In [2]:
def extract_CVDP(IndicatorID, AreaTypeID, TimePeriodID):
    # Initialize dataframes
    raw_data = pd.DataFrame()
    invalid_combinations = pd.DataFrame({
        "timeperiod": [None],
        "system_level": [None],
        "indicatorID": [None],
        "error_message": [None]
    })

    print("This process may take several minutes...")

    # Start of loop
    for i in TimePeriodID:
        print(f" - Extracting data for time period {i}")
        for j in AreaTypeID:
            for k in IndicatorID:
                try:
                    csv_url = f"https://api.cvdprevent.nhs.uk/indicator/{k}/rawDataCSV?timePeriodID={i}&systemLevelID={j}"
                    
                    temp_data = pd.read_csv(csv_url)
                    temp_data["AreaType"] = {
                        1: "CTRY",
                        2: "STP",
                        3: "CCG",
                        4: "PCN",
                        5: "Practice",
                        6: "RGN",
                        7: "ICB",
                        8: "LOC"
                    }.get(j, None)
                    
                    raw_data = pd.concat([raw_data, temp_data], ignore_index=True)

                except Exception as e:
                    error_message = str(e)
                    invalid_combinations = pd.concat([
                        invalid_combinations,
                        pd.DataFrame({
                            "timeperiod": [i],
                            "system_level": [j],
                            "indicatorID": [k],
                            "error_message": [error_message]
                        })
                    ], ignore_index=True)

    print("Extraction completed")

    return {
        "data": raw_data,
        "invalid_combinations": invalid_combinations
    }

# Usage:
# IndicatorID = [...] # fill in the list of Indicator IDs
# AreaTypeID = [...]  # fill in the list of Area Type IDs
# TimePeriodID = [...]  # fill in the list of Time Period IDs
# result = extract_CVDP(IndicatorID, AreaTypeID, TimePeriodID)


In [9]:
# Usage:
IndicatorID = [1] # fill in the list of Indicator IDs
AreaTypeID = [7]  # fill in the list of Area Type IDs
TimePeriodID = [10]  # fill in the list of Time Period IDs

# Call function to extract the data 
result = extract_CVDP(IndicatorID, AreaTypeID, TimePeriodID)

This process may take several minutes...
 - Extracting data for time period 10
Extraction completed


In [32]:
# Access the data element from the result dictionary
raw_data_df = result["data"]

# Load data into database

In [44]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Connection string
connection_string = "mssql+pyodbc://MLCSU-BI-SQL/EAT_Reporting_BSOL?driver=SQL+Server+Native+Client+11.0&trusted_connection=yes"

# Create the database engine
engine = create_engine(connection_string)

In [46]:
# Write the DataFrame to SQL table
table_name = '##cvdprevent' # The name of the table where you want to insert the data
#schema_name = 'YourSchemaName' # Optional: Specify the schema (if applicable)

# SQL command to drop table if it exists
drop_table_command = f"DROP TABLE IF EXISTS {table_name}"

try:
    # Execute the drop table command
    with engine.connect() as connection:
        connection.execute(text(drop_table_command))
        print(f"Table {table_name} dropped successfully (if it existed).")
except SQLAlchemyError as e:
    print(f"An error occurred: {e}")

# Then attempt to write the DataFrame to the SQL table
try:
    raw_data_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
except SQLAlchemyError as e:
    print(f"An error occurred when writing to the table: {e}")

Table ##cvdprevent dropped successfully (if it existed).
An error occurred when writing to the table: Could not reflect: requested table(s) not available in Engine(mssql+pyodbc://MLCSU-BI-SQL/EAT_Reporting_BSOL?driver=SQL+Server+Native+Client+11.0&trusted_connection=yes): (##cvdprevent)


# Get data from database

In [50]:
sql_query = 'SELECT TOP 100 * FROM working.dbo.CVDPrevent_BSOL_S'

df = pd.read_sql_query(sql_query, engine)


In [ ]:
# Pickle the data for later use
pickle_file_path = 'BSOL_1255_CVD_Prevent_Data.pkl' # The name of your pickle file

# Pickle the DataFrame
df.to_pickle(pickle_file_path)